In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import glob
from astropy.io.fits import getdata, getheader, writeto, append, setval

In [2]:
def write_spec(outspec, exp, outname='draft.pha', det_id= 0, mod='A'):
    # Read the template
    null, hdr = getdata('nu30001039002_srcA_sr.pha', 0, header=True)
    writeto(outname, null, header=hdr, overwrite=True)
    spec, hdr = getdata('nu30001039002_srcA_sr.pha', 1, header=True)
    spec['COUNTS'] = outspec
    hdr['RESPFILE'] = 'det{}{}.rmf'.format(det_id, mod)
    hdr['BACKFILE'] = ''
    hdr['ANCRFILE'] = ''
    hdr['EXPOSURE'] = exp
    append(outname, spec, hdr)


In [3]:
def read_spec(infile, det_id=0):
    # returns spec, edges, exposure
    evdata = getdata(infile, 1)

    # Just take DET0 and GRADE==0 for now:
    filter = ( (evdata['GRADE']==0) & (evdata['DET_ID']==det_id) & (evdata['LIMB_ANGLE'] < -2) )
    inds = filter.nonzero()
    
    ehist, edges = np.histogram(evdata['PI'][inds[0]], range = [0, 4096],
                       bins=4096)
    return ehist, edges

In [18]:
def check_coincidence(file1, file2, window=1e-3):
    # Checks for coincidence triggers between FPMA and FPMB within "window" times

    evdata1 = getdata(file1, 1)
    evdata2 = getdata(file2, 1)
    

    nph1 = len(evdata1)
    nph2 = len(evdata2)

    startj = 0
    for i in range(nph1):
        if evdata1['LIMB_ANGLE'][i] > 0:
            continue
        t1 = evdata1['TIME'][i]


        nsearch = (nph2 - startj)
        
        t2_prev = evdata2['TIME'][startj]
        dt_prev = np.abs(t1 - t2_prev)
        if (dt_prev < window):
            print(dt_prev, evdata1['PI'][i] * 0.04 + 1.6, evdata2['PI'][startj]*0.04+1.6, evdata1['LIMB_ANGLE'][i])        

        for j in range(nsearch):
            checkj = j + startj
            if checkj >= nph2:
                break
            t2 = evdata2['TIME'][checkj]
            dt = t2 - t1
            
            if (dt < 0):
                startj = checkj
                continue
            elif (dt < window):
                print(dt, evdata1['PI'][i] * 0.04 + 1.6, evdata2['PI'][checkj]*0.04+1.6, evdata1['LIMB_ANGLE'][i])
            else:
                break
    return

In [19]:
def loop_evts():
    # Returns the full mission data unbinned spectrum
    # set divided into epochs along with exposure.
    
    for ind, evtdir in enumerate(glob.glob('../scripts/reprocess_background/full_mission/*/')):

        evtA = glob.glob('{}/*{}_02.fits'.format(evtdir, 'A'))
        evtB = glob.glob('{}/*{}_02.fits'.format(evtdir, 'B'))

        check_coincidence(evtA[0], evtB[0], window = 10e-6)
        if (ind > 100):
            break

In [20]:
loop_evts()

1.07288360596e-06 27.88 6.4 -11.1076
9.77516174316e-06 28.56 125.64 -22.4552
1.10268592834e-06 115.08 97.04 -19.3386
0.0 43.2 104.04 -30.632
3.24845314026e-06 53.16 89.04 -6.23476
1.07288360596e-06 73.72 25.28 -29.674
3.24845314026e-06 63.28 87.96 -12.9732
4.33623790741e-06 22.92 161.32 -19.2848
2.1755695343e-06 7.0 73.96 -9.94409
3.24845314026e-06 101.32 21.96 -14.0436
9.74535942078e-06 45.08 4.48 -30.6387
3.24845314026e-06 2.72 40.96 -36.5927
3.24845314026e-06 160.24 79.12 -4.66498
7.59959220886e-06 3.44 64.6 -3.00333
6.49690628052e-06 37.28 23.28 -3.97781
4.32133674622e-06 5.36 39.28 -13.3065
2.1755695343e-06 2.08 148.56 -0.676437
5.42402267456e-06 29.16 64.96 -5.78116
0.0 91.76 54.08 -9.98678
3.26335430145e-06 108.48 44.6 -2.98501
